In [1]:
import pandas as pd
rowList = open("./data/train_self_original.txt", "r").readlines()
conversations = {
    "conv_id": []
}
chats = {
    "conv_id": [],
    "buddy1": [],
    "chat_id": [],
    "buddy2": []
}
personals = {
    "persona": [],
    "personaId": [],
    "conv_id": []
}
answers = {
    "conv_id": [],
    "chat_id": [],
    "ans_id": [],
    "answer": []
}
isPersona = lambda rowContext : rowContext.find("your persona:")!=-1


In [2]:
def parseAnswer(conv_id, chat_id, answersContext):
    answersList = answersContext.split("|")
    for i in range(0, len(answersList)):
        answers["conv_id"].append(conv_id)
        answers["chat_id"].append(chat_id)
        answers["ans_id"].append(len(answers["ans_id"])+1)
        answers["answer"].append(answersList[i])        


In [3]:
def parseChat(currentContext, conv_id):
    buddy1EndPointPattern = "\t"
    buddy1EndPoint = currentContext.index(buddy1EndPointPattern)
    buddy2EndPointPattern ="\t\t"
    buddy2EndPoint = currentContext.index(buddy2EndPointPattern)
    buddy1str = currentContext[2:buddy1EndPoint]
    chats["conv_id"].append(conv_id)
    chats["buddy1"].append(buddy1str)
    chats["chat_id"].append(len(chats["chat_id"])+1)
    chats["buddy2"].append(currentContext[buddy1EndPoint+len(buddy1EndPointPattern):buddy2EndPoint])
    answersContext = currentContext[buddy2EndPoint+len(buddy2EndPointPattern):]
    parseAnswer(conv_id, chats["chat_id"][-1], answersContext)
    


In [4]:
def parsePersona(conv_id, persona):
    personals["persona"].append(slicePersona(persona))
    personals["personaId"].append(len(personals["personaId"])+1)
    personals["conv_id"].append(conv_id)

def slicePersona(persona) :
    pattern = "your persona:"
    prefixInd = persona.rindex(pattern)
    retInd = persona.index("\n")
    return persona[prefixInd+len(pattern):retInd]

In [5]:
def parseConversation(i):
    personaLen=0
    personaIt = i
    while(isPersona(rowList[personaIt]) and personaIt < len(rowList)):
        personaLen+=1
        personaIt+=1
    conversations["conv_id"].append(len(conversations["conv_id"])+1)
    
    for persona in rowList[i:i+personaLen] :
        parsePersona(conversations["conv_id"][-1], persona)
    return personaLen-1
    

In [6]:
def parseObjects():
    skipPersona = lambda index, personaLen : index+int(personaLen)
    i = 0
    
    while i < len(rowList):
        if isPersona(rowList[i]) :
            personaLen = parseConversation(i)
            i = skipPersona(i, personaLen)
        else:
            parseChat(rowList[i], conversations["conv_id"][-1])
        i+=1

parseObjects()
convdf = pd.DataFrame(conversations, columns=["conv_id"])
personadf = pd.DataFrame(personals, columns=["conv_id", "persona","personaId"])
chatdf = pd.DataFrame(chats, columns=["conv_id", "buddy1", "chat_id", "buddy2"])
answerdf = pd.DataFrame(answers, columns = ["conv_id", "chat_id", "ans_id", "answer"])

In [7]:
personadf

In [8]:
chatdf

,conv_id,buddy1,chat_id,buddy2
0,1,"hi , how are you doing ? i'm getting ready to ...",1,you must be very fast . hunting is one of my f...
1,1,i am ! for my hobby i like to do canning or so...,2,i also remodel homes when i am not out bow hun...
2,1,that's neat . when i was in high school i plac...,3,that's awesome . do you have a favorite season...
3,1,i do not . but i do have a favorite meat since...,4,what is your favorite meat to eat ?
4,1,i would have to say its prime rib . do you hav...,5,i like chicken or macaroni and cheese .
5,1,do you have anything planned for today ? i th...,6,i am going to watch football . what are you ca...
6,1,i think i will can some jam . do you also pla...,7,if i have time outside of hunting and remodeli...
7,2,"hi , how are you doing today ?",8,i am spending time with my 4 sisters what are ...
8,2,"wow , four sisters . just watching game of thr...",9,that is a good show i watch that while drinkin...
9,2,i agree . what do you do for a living ?,10,i'm a researcher i'm researching the fact that...


In [10]:
answerdf

,conv_id,chat_id,ans_id,answer
0,1,1,1,"my mom was single with 3 boys , so we never le..."
1,1,1,2,i try to wear all black every day . it makes m...
2,1,1,3,well nursing stresses you out so i wish luck w...
3,1,1,4,yeah just want to pick up nba nfl getting old
4,1,1,5,i really like celine dion . what about you ?
5,1,1,6,no . i live near farms .
6,1,1,7,"i wish i had a daughter , i'm a boy mom . they..."
7,1,1,8,yeah when i get bored i play gone with the win...
8,1,1,9,hi how are you ? i'm eating dinner with my hub...
9,1,1,10,were you married to your high school sweethear...
